In [1]:
import os
import yaml
import pandas as pd
from pathlib import Path
from matplotlib import pyplot as plt
import seaborn as sns
sns.set()

os.chdir("../scripts")

import preprocess, train, inference, interpret

In [2]:
with open("../model/params.yaml", "r") as params_file:
    params = yaml.safe_load(params_file)

data_dir = params['data_dir']

In [3]:
file_name = "Trepn_2022.04.04_191744_q720_total.csv"
data = preprocess.load_data(file_name)
data.head(3)

,Time,Min,Memory Usage,CPU Frequency,CPU Load,Wi-Fi State,Wi-Fi RSSI Level,Screen Bright,Battery,GPS State,Quality,Download,Upload
0,146,00:00:00,1824916,1586000,40,3,-57,4,71,1,720,820,11
1,10055,00:00:10,1824984,1586000,40,3,-57,4,71,1,720,820,11
2,20038,00:00:20,1826428,902000,40,3,-57,4,71,1,720,820,11


In [4]:
train_df, test_df = preprocess.prep_data(df=data, train_frac=0.6, plot_df=True)

Starting with data preparation...
Completed.


In [5]:

print(test_df.shape)
test_df.head(10)

(93, 10)


,Time,CPU Frequency,CPU Load,Memory Usage,Wi-Fi RSSI Level,Quality,Screen Bright,Battery,Download,Upload
139,1.007247,0.775000,0.0,0.746463,0.3125,0.0,1.0,0.000000,1.0,0.0
140,1.014494,1.000000,0.0,0.676973,0.6250,0.0,1.0,0.000000,1.0,0.0
141,1.021740,1.000000,0.0,0.743841,0.4375,0.0,1.0,0.000000,1.0,0.0
142,1.028987,1.000000,0.0,0.536379,0.4375,0.0,1.0,0.000000,1.0,0.0
143,1.036234,0.342308,0.0,0.722150,0.7500,0.0,1.0,-0.166667,1.0,0.0
144,1.043481,0.450000,0.0,0.316886,0.1875,0.0,1.0,-0.166667,1.0,0.0
145,1.050728,1.000000,0.0,0.563456,0.5625,0.0,1.0,-0.166667,1.0,0.0
146,1.057975,0.342308,0.0,0.734504,0.5625,0.0,1.0,-0.166667,1.0,0.0
147,1.065221,0.342308,0.0,0.250700,0.3750,0.0,1.0,-0.166667,1.0,0.0
148,1.072468,0.342308,0.0,0.574373,0.7500,0.0,1.0,-0.166667,1.0,0.0


In [6]:
sequence_length = 10
batch_size = 2
n_epochs = 50
n_epochs_stop = 10

#sequence_length = 10
#batch_size = 5
#n_epochs = 200
#n_epochs_stop = 50
label_name = 'Battery'

In [7]:
predictions_descaled, labels_descaled = inference.predict(df=test_df,
                                                          label_name='Battery',
                                                          sequence_length=sequence_length)

predictions_descaled.shape, predictions_descaled.shape

print(test_df.shape)

RuntimeError: [enforce fail at inline_container.cc:115] . file in archive is not in a subdirectory: arr_0.npy

In [ ]:
print('Error on all test data:')
inference.print_loss_metrics(labels_descaled, predictions_descaled)
print('\n')
print('Error on partial test data:')
inference.print_loss_metrics(labels_descaled[:30], predictions_descaled[:30])

In [ ]:
# plot predictions vs true values
df_pred = pd.DataFrame()
df_pred['predicted'] = predictions_descaled
df_pred['true'] = labels_descaled
df_pred['residual'] = labels_descaled - predictions_descaled
df_pred.plot(figsize=(12,6), cmap='viridis')
#plt.xticks(rotation=45)
#plt.xlim(0,300)
plt.xlabel('Min')
plt.ylabel('Baterry Level')
plt.legend()
plt.show()

In [ ]:
from train import TimeSeriesDataset, TSModel
from torch.utils.data import DataLoader
import torch
import shap

In [ ]:
background_data_size = 232
test_sample_size = 40
sequence_length = 10